In [1]:
import numpy as np
import face_recognition
import os
import shutil
import random
import tkinter as tk
from tkinter import filedialog, messagebox
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
print("Works")

Works


In [2]:
import cv2
import time
import os
import random
import threading
import string
import pickle
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import shutil
import dlib
import re
import numpy as np
import face_recognition
from tkinter import *
from scipy.spatial.distance import cosine
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
from matplotlib import pyplot as plt
import PIL.Image, PIL.ImageTk
from sklearn.cluster import DBSCAN
from queue import Queue
import matplotlib.animation as animation
from threading import Thread
from PIL import Image, ImageTk
from tkinter import filedialog
from tkinter import simpledialog, messagebox, filedialog
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from tkinter import messagebox, Tk, Label, Button
import tkinter as tk
import uuid
from matplotlib.animation import FuncAnimation
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine
print("Dependencies functional!")

Dependencies functional!


In [3]:
import cv2
import dlib
import os
import shutil
import uuid
import numpy as np
import json 
from tkinter import Tk, Button, filedialog, messagebox
import face_recognition
from datetime import datetime
from collections import defaultdict
import threading

# Paths
detected_faces_folder = "C:/Users/Dennis/Desktop/Repositories/detected_faces"
sorted_faces_folder = "C:/Users/Dennis/Desktop/Repositories/sorted_faces"
detected_faces_info_folder = "C:/Users/Dennis/Desktop/Repositories/detected_faces/detected_faces_info"

# Ensure the directories exist
os.makedirs(detected_faces_folder, exist_ok=True)
os.makedirs(sorted_faces_folder, exist_ok=True)
os.makedirs(detected_faces_info_folder, exist_ok=True)

# Initialize dlib and OpenCV
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("C:/Users/Dennis/Desktop/Repositories/shape_predictor_68_face_landmarks.dat")
face_recognition_model = dlib.face_recognition_model_v1("C:/Users/Dennis/Desktop/Repositories/dlib_face_recognition_resnet_model_v1.dat")

# Tkinter window setup
root = Tk()
root.title("Face Recognition System")

# Global variables for training data and pause functionality
counter = 0
paused = False
capturing = True
name = ""

# Function to capture training data (takes images continuously)
def take_training_data():
    global counter, paused, capturing, name

    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        messagebox.showerror("Error", "Camera not accessible.")
        return

    while capturing:
        if paused:
            continue  # Pause the capture if the flag is set

        ret, frame = cap.read()
        if not ret:
            messagebox.showerror("Error", "Failed to capture frame.")
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        faces = detector(rgb_frame)

        for face in faces:
            # Draw a green rectangle around the face
            cv2.rectangle(frame, (face.left(), face.top()), (face.right(), face.bottom()), (0, 255, 0), 2)

            # Get the facial landmarks and compute the face descriptor
            shape = predictor(rgb_frame, face)
            face_descriptor = face_recognition_model.compute_face_descriptor(rgb_frame, shape)
            face_descriptor = np.array(face_descriptor)

            # If counter is 0 (first image), generate a random UUID for the folder and image names
            if counter == 0 and not name:
                name = str(uuid.uuid4())  # Generate a random UUID for the folder and image name

            # Create a folder with the random UUID name for storing images if it doesn't exist
            person_folder = os.path.join(detected_faces_folder, name)
            os.makedirs(person_folder, exist_ok=True)

            # Capture face image and save it with a random name
            face_image = frame[face.top():face.bottom(), face.left():face.right()]
            unique_image_name = f"{uuid.uuid4()}.jpg"  # Generate a unique name for each image
            cv2.imwrite(os.path.join(person_folder, unique_image_name), face_image)

            # Save face descriptor info
            face_info = {
                "name": name,
                "descriptor": face_descriptor.tolist(),
            }
            with open(os.path.join(detected_faces_info_folder, f"{name}_{counter}_info.json"), "w") as f:
                json.dump(face_info, f)

            counter += 1
            cv2.putText(frame, f"Captured {counter}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Show the webcam feed
        cv2.imshow("Capturing Face Data", frame)

        # Exit the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    if counter > 0:
        messagebox.showinfo("Training Complete", f"{counter} images captured successfully.")
        
# Function to toggle the pause/resume functionality
def pause_capture():
    global paused
    paused = not paused  # Toggle the pause state
    if paused:
        print("Capture paused.")
    else:
        print("Capture resumed.")

# Function to stop the camera capture completely
def stop_capture():
    global capturing
    capturing = False  # Stop the camera capture loop
    print("Capture stopped.")
    cv2.destroyAllWindows()

# Function for facial recognition
def start_recognition():
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        messagebox.showerror("Error", "Camera not accessible.")
        return

    known_face_encodings = []
    known_face_names = []

    # Load the sorted face encodings
    for folder_name in os.listdir(sorted_faces_folder):
        folder_path = os.path.join(sorted_faces_folder, folder_name)
        if os.path.isdir(folder_path):
            for file_name in os.listdir(folder_path):
                if file_name.endswith(".jpg"):
                    image_path = os.path.join(folder_path, file_name)
                    image = face_recognition.load_image_file(image_path)
                    face_locations = face_recognition.face_locations(image)
                    face_encodings = face_recognition.face_encodings(image, face_locations)
                    if face_encodings:
                        known_face_encodings.append(face_encodings[0])
                        known_face_names.append(folder_name)

    while True:
        ret, frame = cap.read()
        if not ret:
            messagebox.showerror("Error", "Failed to capture frame.")
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        faces = detector(rgb_frame)

        for face in faces:
            # Draw rectangle around the face
            cv2.rectangle(frame, (face.left(), face.top()), (face.right(), face.bottom()), (0, 255, 0), 2)

            # Get the facial landmarks and compute the face descriptor
            shape = predictor(rgb_frame, face)
            face_descriptor = face_recognition_model.compute_face_descriptor(rgb_frame, shape)
            face_descriptor = np.array(face_descriptor)

            # Compare with known face encodings
            matches = face_recognition.compare_faces(known_face_encodings, face_descriptor)
            distances = face_recognition.face_distance(known_face_encodings, face_descriptor)
            best_match_index = np.argmin(distances)

            if matches[best_match_index]:
                name = known_face_names[best_match_index]
                cv2.putText(frame, f"Name: {name}", (face.left(), face.top() - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
                cv2.putText(frame, f"Distance: {distances[best_match_index]:.2f}", (face.left(), face.bottom() + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            
        # Show the webcam feed
        cv2.imshow("Facial Recognition", frame)

        # Exit the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

def sort_files():
    # Open a file dialog to allow the user to select images to sort
    file_paths = filedialog.askopenfilenames(
        title="Select Images to Sort",
        initialdir=detected_faces_folder,
        filetypes=[("Image Files", "*.jpg *.png")]
    )

    if not file_paths:
        messagebox.showinfo("No files selected", "No files were selected to sort.")
        return

    # Process and encode faces from the selected images
    encodings = []
    for image_path in file_paths:
        image = face_recognition.load_image_file(image_path)
        face_locations = face_recognition.face_locations(image)
        face_encodings = face_recognition.face_encodings(image, face_locations)  # Get encodings for each face

        if face_encodings:
            encodings.extend(face_encodings)  # Add all detected faces
        else:
            print(f"Warning: No face detected in {image_path}")

    # Load existing images from the "sorted_faces" folder and encode them
    existing_encodings = defaultdict(list)
    for folder_name in os.listdir(sorted_faces_folder):
        folder_path = os.path.join(sorted_faces_folder, folder_name)
        if os.path.isdir(folder_path):
            for file_name in os.listdir(folder_path):
                if file_name.endswith(".jpg"):
                    saved_image = face_recognition.load_image_file(os.path.join(folder_path, file_name))
                    saved_encoding = face_recognition.face_encodings(saved_image)
                    if saved_encoding:
                        existing_encodings[folder_name].append(saved_encoding[0])

    # Now compare the new images with the existing ones
    for i, encoding in enumerate(encodings):
        matched_folder = None
        for folder_name, saved_encodings in existing_encodings.items():
            for saved_encoding in saved_encodings:
                distance = face_recognition.face_distance([saved_encoding], encoding)
                if distance < 0.4:  # Set a stricter threshold for matching
                    matched_folder = folder_name
                    break

            if matched_folder:
                break

        if matched_folder:
            # If a match was found, move the image into the corresponding folder
            cluster_folder = os.path.join(sorted_faces_folder, matched_folder)

            # Ensure the file name is unique by appending a timestamp or UUID
            unique_name = f"{datetime.now().strftime('%Y%m%d%H%M%S')}_{os.path.basename(file_paths[i])}"
            try:
                shutil.copy(file_paths[i], os.path.join(cluster_folder, unique_name))
                print(f"Image {file_paths[i]} added to {matched_folder}")
            except Exception as e:
                print(f"Error copying file {file_paths[i]}: {e}")
        else:
            # If no match was found, create a new folder and add the image there
            new_folder = f"new_person_{datetime.now().strftime('%Y%m%d%H%M%S')}"
            new_folder_path = os.path.join(sorted_faces_folder, new_folder)
            os.makedirs(new_folder_path, exist_ok=True)

            # Ensure the file name is unique
            unique_name = f"{datetime.now().strftime('%Y%m%d%H%M%S')}_{os.path.basename(file_paths[i])}"
            try:
                shutil.copy(file_paths[i], os.path.join(new_folder_path, unique_name))
                print(f"Image {file_paths[i]} added to new folder {new_folder}")
            except Exception as e:
                print(f"Error copying file {file_paths[i]}: {e}")

    print("Files sorted successfully.")

# Tkinter Button for training data
button_train = Button(root, text="Take Training Data", command=take_training_data)
button_train.pack(pady=10)

# Tkinter Button for sorting images
button_sort = Button(root, text="Sort Files", command=sort_files)
button_sort.pack(pady=10)

# Tkinter Button for starting recognition
button_recognition = Button(root, text="Facial Recognition", command=start_recognition)
button_recognition.pack(pady=10)

# Tkinter Button for pausing/resuming image capture
button_pause = Button(root, text="Pause/Resume Capture", command=pause_capture)
button_pause.pack(pady=10)

# Tkinter Button for stopping image capture
button_stop = Button(root, text="Stop Camera", command=stop_capture)
button_stop.pack(pady=10)

# Run Tkinter main loop
root.mainloop()


Image C:/Users/Dennis/Desktop/Repositories/detected_faces/88ea70cb-704a-4a25-86a3-ab56d2b8be50/0d632b62-4d6a-46d8-b36d-36a180a5c902.jpg added to new folder new_person_20250321113227
Image C:/Users/Dennis/Desktop/Repositories/detected_faces/88ea70cb-704a-4a25-86a3-ab56d2b8be50/1b0ae4de-b5f6-450c-b5ec-81e14e075b69.jpg added to new folder new_person_20250321113227
Image C:/Users/Dennis/Desktop/Repositories/detected_faces/88ea70cb-704a-4a25-86a3-ab56d2b8be50/1b5d12b4-0a96-4d59-b526-bd3611d1e6aa.jpg added to new folder new_person_20250321113227
Image C:/Users/Dennis/Desktop/Repositories/detected_faces/88ea70cb-704a-4a25-86a3-ab56d2b8be50/1d4eb9da-3f5f-4b18-887e-37d12d6fecd7.jpg added to new folder new_person_20250321113227
Image C:/Users/Dennis/Desktop/Repositories/detected_faces/88ea70cb-704a-4a25-86a3-ab56d2b8be50/1e7635b1-150b-47bb-98ec-bc8f99d0cc51.jpg added to new folder new_person_20250321113227
Image C:/Users/Dennis/Desktop/Repositories/detected_faces/88ea70cb-704a-4a25-86a3-ab56d2b8